# Robotino Charging Simulation Notebook
This notebook explains the Python script used for simulating the charging process of multiple Robotino robots. Each section contains a detailed explanation of the code followed by the corresponding implementation.

### Import Statements
The necessary libraries are imported here. The `queue` module is used for priority management, and the `time` module is used to simulate delays in the charging process.

In [1]:
from queue import PriorityQueue
import time

### Robotino Class
This class defines the behavior of a Robotino robot, including its attributes such as:
- `robot_id`: Unique identifier for the robot.
- `version`: Version of the Robotino.
- `battery_level`: Current battery level, default is 100%.

Methods:
- `drain_battery(amount)`: Reduces the battery by the specified amount.
- `start_charging()`: Marks the Robotino as charging.
- `stop_charging()`: Marks the Robotino as not charging.
- `charge_battery(amount)`: Increases the battery level if charging.

In [ ]:
class Robotino:
    def __init__(self, robot_id, version, battery_level=100):
        self.robot_id = robot_id
        self.version = version
        self.battery_level = battery_level
        self.is_charging = False

    def drain_battery(self, amount=10):
        if not self.is_charging:
            self.battery_level = max(0, self.battery_level - amount)

    def start_charging(self):
        self.is_charging = True

    def stop_charging(self):
        self.is_charging = False

    def charge_battery(self, amount=10):
        if self.is_charging:
            self.battery_level = min(100, self.battery_level + amount)

### Charger Class
This class represents a charging station for Robotinos. It includes attributes such as:
- `charger_id`: Unique identifier for the charger.
- `occupied`: Indicates if the charger is in use.
- `current_robotino`: References the Robotino currently being charged.

Methods:
- `connect_robotino(robotino)`: Connects a Robotino to the charger.
- `disconnect_robotino()`: Disconnects the Robotino from the charger.
- `charge_robotino(amount)`: Charges the connected Robotino by the specified amount.

In [ ]:
class Charger:
    def __init__(self, charger_id):
        self.charger_id = charger_id
        self.occupied = False
        self.current_robotino = None

    def connect_robotino(self, robotino):
        if not self.occupied:
            self.occupied = True
            self.current_robotino = robotino
            robotino.start_charging()

    def disconnect_robotino(self):
        if self.occupied and self.current_robotino:
            self.current_robotino.stop_charging()
            self.current_robotino = None
            self.occupied = False

    def charge_robotino(self, amount=10):
        if self.occupied and self.current_robotino:
            self.current_robotino.charge_battery(amount)

### Simulation Function
The `simulate_charging_extended` function manages the simulation. It involves:
- Draining battery levels for Robotinos.
- Allocating chargers to low-battery Robotinos.
- Charging the connected Robotinos.
- Disconnecting fully charged Robotinos.

Key parameters include:
- `cycles`: Number of simulation cycles.
- `drain_amount`: Battery drain per cycle.
- `charge_amount`: Battery charge per cycle.
- `low_battery_threshold`: Threshold below which Robotinos are prioritized for charging.

In [ ]:
def simulate_charging_extended(robotinos, chargers, cycles=10, drain_amount=15, charge_amount=20, low_battery_threshold=30):
    robotino_lookup = {robotino.robot_id: robotino for robotino in robotinos}
    total_cycles_charged = {robotino.robot_id: 0 for robotino in robotinos}

    for cycle in range(cycles):
        print(f"Cycle {cycle + 1}:")
        low_battery_queue = PriorityQueue()

        for robotino in robotinos:
            robotino.drain_battery(drain_amount)
            print(f"{robotino.robot_id} battery level: {robotino.battery_level}%")
            if robotino.battery_level < low_battery_threshold and not robotino.is_charging:
                robot_id_num = int(robotino.robot_id[1:])
                low_battery_queue.put((robotino.battery_level, robot_id_num))

        while not low_battery_queue.empty():
            _, robot_id_num = low_battery_queue.get()
            robot_id = f"R{robot_id_num}"
            robotino = robotino_lookup[robot_id]

            allocated = False
            for charger in chargers:
                if not charger.occupied:
                    charger.connect_robotino(robotino)
                    print(f"{robotino.robot_id} connected to {charger.charger_id}")
                    allocated = True
                    break
            if not allocated:
                print(f"No available charger for {robotino.robot_id}. It remains in queue.")

        for charger in chargers:
            charger.charge_robotino(charge_amount)
            if charger.occupied:
                total_cycles_charged[charger.current_robotino.robot_id] += 1
                print(f"{charger.current_robotino.robot_id} is charging on {charger.charger_id}, battery level: {charger.current_robotino.battery_level}%")

        for charger in chargers:
            if charger.occupied and charger.current_robotino.battery_level >= 100:
                print(f"{charger.current_robotino.robot_id} fully charged and disconnecting from {charger.charger_id}")
                charger.disconnect_robotino()

        time.sleep(1)
        print("\n" + "=" * 40 + "\n")

    print("Simulation complete. Summary of charging cycles per Robotino:")
    for robot_id, cycles_charged in total_cycles_charged.items():
        print(f"{robot_id} spent {cycles_charged} cycles in charging.")

### Initialization and Execution
Here, Robotinos and chargers are initialized, and the simulation function is executed.

In [ ]:
robotinos = [Robotino(robot_id=f"R{i + 1}", version=3) for i in range(5)]
robotinos.append(Robotino(robot_id="R6", version=4))
chargers = [Charger(charger_id=f"C{i + 1}") for i in range(6)]

simulate_charging_extended(robotinos, chargers)